In [10]:
## import packages

import requests
import pandas as pd
import io, json, time

In [12]:
# define a dictionary of NOAA state codes 1–51 (except 49 = Pacific Islands) based on Climate at a Glance page to fit API parameters
STATE_CODES = {
    1:"Alabama", 2:"Arizona", 3:"Arkansas", 4:"California", 5:"Colorado",
    6:"Connecticut", 7:"Delaware", 8:"Florida", 9:"Georgia", 10:"Idaho",
    11:"Illinois", 12:"Indiana", 13:"Iowa", 14:"Kansas", 15:"Kentucky",
    16:"Louisiana", 17:"Maine", 18:"Maryland", 19:"Massachusetts", 20:"Michigan",
    21:"Minnesota", 22:"Mississippi", 23:"Missouri", 24:"Montana", 25:"Nebraska",
    26:"Nevada", 27:"New Hampshire", 28:"New Jersey", 29:"New Mexico", 30:"New York",
    31:"North Carolina", 32:"North Dakota", 33:"Ohio", 34:"Oklahoma", 35:"Oregon",
    36:"Pennsylvania", 37:"Rhode Island", 38:"South Carolina", 39:"South Dakota",
    40:"Tennessee", 41:"Texas", 42:"Utah", 43:"Vermont", 44:"Virginia",
    45:"Washington", 46:"West Virginia", 47:"Wisconsin", 48:"Wyoming",
    50:"Alaska", 51:"Hawaii"
}

## define minimum and maximum temperature as search parameters, going back 100 years from 2025
PARAMS = ["tmax", "tmin"]
START_YEAR = 1925
END_YEAR = 2025   
BASE_URL = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series"

def get_state_param(state_id, param):
    url = f"{BASE_URL}/{state_id}/{param}/1/0/{START_YEAR}-{END_YEAR}.csv"
    r = requests.get(url, timeout=60)
    if r.status_code != 200:
        print(f"Failed to get {param} for state {state_id}: {r.status_code}")
        return []
        
    lines = [ln for ln in r.text.splitlines() if not ln.startswith("#")]
    if len(lines) < 2:
        print(f"No data found for {param} {state_id}")
        return []

    df = pd.read_csv(io.StringIO("\n".join(lines)))
    if "Date" not in df.columns or "Value" not in df.columns:
        print(f"Unexpected CSV structure for {param} {state_id}")
        return []

    records = []
    for _, row in df.iterrows():
        try:
            date_str = str(int(row["Date"]))  # ex) 192501
            year = int(date_str[:4])
            month = int(date_str[4:6])
            value = float(row["Value"])
            records.append({"year": year, "month": month, "value": value})
        except Exception:
            continue
    return records

def main():
    data = {}
    for state_id, state_name in STATE_CODES.items():
        print(f"Getting {state_name}...")
        state_entry = {}
        for param in PARAMS:
            state_entry[param] = get_state_param(state_id, param)
            time.sleep(0.5)  # pause
        data[state_name] = state_entry

    with open("state_temps.json", "w") as f:
        json.dump(data, f, indent=2)
    print("Saved noaa_state_temperature_1925_present.json")

if __name__ == "__main__":
    main()


Getting Alabama...
Getting Arizona...
Getting Arkansas...
Getting California...
Getting Colorado...
Getting Connecticut...
Getting Delaware...
Getting Florida...
Getting Georgia...
Getting Idaho...
Getting Illinois...
Getting Indiana...
Getting Iowa...
Getting Kansas...
Getting Kentucky...
Getting Louisiana...
Getting Maine...
Getting Maryland...
Getting Massachusetts...
Getting Michigan...
Getting Minnesota...
Getting Mississippi...
Getting Missouri...
Getting Montana...
Getting Nebraska...
Getting Nevada...
Getting New Hampshire...
Getting New Jersey...
Getting New Mexico...
Getting New York...
Getting North Carolina...
Getting North Dakota...
Getting Ohio...
Getting Oklahoma...
Getting Oregon...
Getting Pennsylvania...
Getting Rhode Island...
Getting South Carolina...
Getting South Dakota...
Getting Tennessee...
Getting Texas...
Getting Utah...
Getting Vermont...
Getting Virginia...
Getting Washington...
Getting West Virginia...
Getting Wisconsin...
Getting Wyoming...
Getting Alaska

In [14]:
## check that it works 

state = "Alaska"
param = "tmin"
data[state][param][:10] 

NameError: name 'data' is not defined

In [15]:
import requests
import pandas as pd
import json
import io
import time

# NOAA state codes 1–51 (except 49 = Pacific Islands)
STATE_CODES = {
    1:"Alabama", 2:"Arizona", 3:"Arkansas", 4:"California", 5:"Colorado",
    6:"Connecticut", 7:"Delaware", 8:"Florida", 9:"Georgia", 10:"Idaho",
    11:"Illinois", 12:"Indiana", 13:"Iowa", 14:"Kansas", 15:"Kentucky",
    16:"Louisiana", 17:"Maine", 18:"Maryland", 19:"Massachusetts", 20:"Michigan",
    21:"Minnesota", 22:"Mississippi", 23:"Missouri", 24:"Montana", 25:"Nebraska",
    26:"Nevada", 27:"New Hampshire", 28:"New Jersey", 29:"New Mexico", 30:"New York",
    31:"North Carolina", 32:"North Dakota", 33:"Ohio", 34:"Oklahoma", 35:"Oregon",
    36:"Pennsylvania", 37:"Rhode Island", 38:"South Carolina", 39:"South Dakota",
    40:"Tennessee", 41:"Texas", 42:"Utah", 43:"Vermont", 44:"Virginia",
    45:"Washington", 46:"West Virginia", 47:"Wisconsin", 48:"Wyoming",
    50:"Alaska", 51:"Hawaii"
}

# Climate Impact Lab GEOID to state name mapping
GEOID_TO_STATE = {
    '1': 'Alabama', '2': 'Alaska', '4': 'Arizona', '5': 'Arkansas',
    '6': 'California', '8': 'Colorado', '9': 'Connecticut', '10': 'Delaware',
    '11': 'District of Columbia', '12': 'Florida', '13': 'Georgia', '15': 'Hawaii',
    '16': 'Idaho', '17': 'Illinois', '18': 'Indiana', '19': 'Iowa',
    '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana', '23': 'Maine',
    '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska',
    '32': 'Nevada', '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico',
    '36': 'New York', '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio',
    '40': 'Oklahoma', '41': 'Oregon', '42': 'Pennsylvania', '44': 'Rhode Island',
    '45': 'South Carolina', '46': 'South Dakota', '47': 'Tennessee', '48': 'Texas',
    '49': 'Utah', '50': 'Vermont', '51': 'Virginia', '53': 'Washington',
    '54': 'West Virginia', '55': 'Wisconsin', '56': 'Wyoming',
    '60': 'American Samoa', '66': 'Guam', '69': 'Commonwealth of the Northern Mariana Islands',
    '72': 'Puerto Rico', '78': 'United States Virgin Islands'
}

PARAMS = ["tavg", "tmax", "tmin"]
START_YEAR = 1925
END_YEAR = 2025
BASE_URL = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series"

def get_state_param(state_id, param):
    """Fetch historical temperature data from NOAA"""
    url = f"{BASE_URL}/{state_id}/{param}/1/0/{START_YEAR}-{END_YEAR}.csv"
    r = requests.get(url, timeout=60)
    if r.status_code != 200:
        print(f"Failed to get {param} for state {state_id}: {r.status_code}")
        return []
        
    lines = [ln for ln in r.text.splitlines() if not ln.startswith("#")]
    if len(lines) < 2:
        print(f"No data found for {param} {state_id}")
        return []
    
    df = pd.read_csv(io.StringIO("\n".join(lines)))
    if "Date" not in df.columns or "Value" not in df.columns:
        print(f"Unexpected CSV structure for {param} {state_id}")
        return []
    
    records = []
    for _, row in df.iterrows():
        try:
            date_str = str(int(row["Date"]))  # ex) 192501
            year = int(date_str[:4])
            month = int(date_str[4:6])
            val = float(row["Value"])
            records.append({"year": year, "month": month, "value": val})
        except Exception:
            continue
    return records

def get_future_temps():
    """Extract future temperature projections from CSV"""
    df = pd.read_csv('ClimateImpactLab_USData_20March2023(4).csv', skiprows=1)
    df.columns = df.columns.str.strip()
    
    future_temps = {}
    for _, row in df.iterrows():
        fips_code = str(int(row['GEOID'])).zfill(2)
        state_name = FIPS_TO_STATE.get(fips_code)
        
        if state_name:
            # Get midcentury and end of century temperatures (0.50 column)
            midcentury_temp = round(row.iloc[5], 2)  # 2040-2059
            end_century_temp = round(row.iloc[11], 2)  # 2080-2099
            
            future_temps[state_name] = {
                'midcentury': midcentury_temp,
                'end_century': end_century_temp
            }
    
    return future_temps

def main():
    # Get historical data from NOAA
    print("Fetching historical data from NOAA...")
    data = {}
    for sid, sname in STATE_CODES.items():
        print(f"Getting {sname}...")
        state_entry = {}
        for param in PARAMS:
            state_entry[param] = get_state_param(sid, param)
            time.sleep(0.5)  # pause between requests
        data[sname] = state_entry
    
    # Get future temperature projections
    print("\nAdding future temperature projections...")
    future_temps = get_future_temps()
    
    # Add future projections to each state's tavg data
    for state_name, temps in future_temps.items():
        if state_name in data:
            # Add midcentury projection (year 2040)
            data[state_name]['tavg'].append({
                "year": 2040,
                "month": 0,  # 0 indicates annual average
                "value": temps['midcentury']
            })
            
            # Add end of century projection (year 2080)
            data[state_name]['tavg'].append({
                "year": 2080,
                "month": 0,  # 0 indicates annual average
                "value": temps['end_century']
            })
            
            print(f"Added future temps for {state_name}: 2040={temps['midcentury']}°F, 2080={temps['end_century']}°F")
    
    # Save combined data
    with open("state_temps_with_projections.json", "w") as f:
        json.dump(data, f, indent=2)
    
    print("\nSuccessfully saved state_temps_with_projections.json")

if __name__ == "__main__":
    main()

Fetching historical data from NOAA...
Getting Alabama...
Getting Arizona...
Getting Arkansas...
Getting California...
Getting Colorado...
Getting Connecticut...
Getting Delaware...
Getting Florida...
Getting Georgia...
Getting Idaho...
Getting Illinois...
Getting Indiana...
Getting Iowa...
Getting Kansas...
Getting Kentucky...
Getting Louisiana...
Getting Maine...
Getting Maryland...
Getting Massachusetts...
Getting Michigan...
Getting Minnesota...
Getting Mississippi...
Getting Missouri...
Getting Montana...
Getting Nebraska...
Getting Nevada...
Getting New Hampshire...
Getting New Jersey...
Getting New Mexico...
Getting New York...
Getting North Carolina...
Getting North Dakota...
Getting Ohio...
Getting Oklahoma...
Getting Oregon...
Getting Pennsylvania...
Getting Rhode Island...
Getting South Carolina...
Getting South Dakota...
Getting Tennessee...
Getting Texas...
Getting Utah...
Getting Vermont...
Getting Virginia...
Getting Washington...
Getting West Virginia...
Getting Wisconsi

ValueError: cannot convert float NaN to integer

In [36]:
import pandas as pd
import json


In [38]:
## Days over 35C

In [40]:
# Read Excel
df = pd.read_excel("hot.xlsx")

# Convert to dictionary
heat_data = {}
for _, row in df.iterrows():
    heat_data[row['STATE']] = {
        "extreme_heat_days": {
            "2005": round(float(row['B2005'])),
            "2039": round(float(row['B2039'])),
            "2059": round(float(row['B2059'])),
            "2099": round(float(row['B2099']))
        }
    }
print(heat_data)
# Save to JSON
with open('extreme_heat_days.json', 'w') as f:
    json.dump(heat_data, f, indent=2)


{'Alabama': {'extreme_heat_days': {'2005': 7, '2039': 16, '2059': 23, '2099': 51}}, 'Alaska': {'extreme_heat_days': {'2005': 0, '2039': 0, '2059': 0, '2099': 0}}, 'Arizona': {'extreme_heat_days': {'2005': 102, '2039': 120, '2059': 130, '2099': 153}}, 'Arkansas': {'extreme_heat_days': {'2005': 16, '2039': 31, '2059': 42, '2099': 75}}, 'California': {'extreme_heat_days': {'2005': 11, '2039': 18, '2059': 22, '2099': 35}}, 'Colorado': {'extreme_heat_days': {'2005': 2, '2039': 5, '2059': 10, '2099': 22}}, 'Connecticut': {'extreme_heat_days': {'2005': 1, '2039': 2, '2059': 3, '2099': 8}}, 'Delaware': {'extreme_heat_days': {'2005': 2, '2039': 4, '2059': 6, '2099': 16}}, 'District of Columbia': {'extreme_heat_days': {'2005': 5, '2039': 11, '2059': 17, '2099': 37}}, 'Florida': {'extreme_heat_days': {'2005': 2, '2039': 5, '2059': 9, '2099': 32}}, 'Georgia': {'extreme_heat_days': {'2005': 7, '2039': 15, '2059': 21, '2099': 45}}, 'Hawaii': {'extreme_heat_days': {'2005': 0, '2039': 0, '2059': 0, '2

In [42]:
## Days under 0C

In [44]:
# Read Excel
df = pd.read_excel("Cold.xlsx")

# Convert to dictionary
cold_data = {}
for _, row in df.iterrows():
    cold_data[row['STATE']] = {
        "extreme_cold_days": {
            "2005": round(float(row['B2005'])),
            "2039": round(float(row['B2039'])),
            "2059": round(float(row['B2059'])),
            "2099": round(float(row['B2099']))
        }
    }
print(cold_data)
# Save to JSON
with open('extreme_cold_days.json', 'w') as f:
    json.dump(cold_data, f, indent=2)

{'Alabama': {'extreme_cold_days': {'2005': 31, '2039': 23, '2059': 21, '2099': 13}}, 'Alaska': {'extreme_cold_days': {'2005': 193, '2039': 170, '2059': 161, '2099': 133}}, 'Arizona': {'extreme_cold_days': {'2005': 12, '2039': 9, '2059': 8, '2099': 5}}, 'Arkansas': {'extreme_cold_days': {'2005': 50, '2039': 39, '2059': 35, '2099': 23}}, 'California': {'extreme_cold_days': {'2005': 4, '2039': 2, '2059': 2, '2099': 1}}, 'Colorado': {'extreme_cold_days': {'2005': 171, '2039': 156, '2059': 145, '2099': 124}}, 'Connecticut': {'extreme_cold_days': {'2005': 93, '2039': 75, '2059': 64, '2099': 46}}, 'Delaware': {'extreme_cold_days': {'2005': 60, '2039': 47, '2059': 38, '2099': 23}}, 'District of Columbia': {'extreme_cold_days': {'2005': 66, '2039': 52, '2059': 42, '2099': 25}}, 'Florida': {'extreme_cold_days': {'2005': 2, '2039': 1, '2059': 1, '2099': 0}}, 'Georgia': {'extreme_cold_days': {'2005': 36, '2039': 27, '2059': 24, '2099': 15}}, 'Hawaii': {'extreme_cold_days': {'2005': 0, '2039': 0, '